In [1]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils
using LinearAlgebra

# Plots
using CairoMakie

In [2]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 1
const n_levels = 2

t_f = 10 * ns
n_steps = 101
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

0.1

In [3]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
X0=kron(Paulis["X"],Paulis["I"])
Y0=kron(Paulis["Y"],Paulis["I"])
X1=kron(Paulis["I"],Paulis["X"])
Y1=kron(Paulis["I"],Paulis["Y"])

excitation(theta) = exp(-im/2 * theta * (X0 * Y1-Y0 * X1))

excitation (generic function with 1 method)

In [4]:
a = [0 1; 0 0]
ad = transpose(a)
a0  = kron(a,Paulis["I"])
ad0  = kron(ad,Paulis["I"])
a1  = kron(Paulis["I"],a)
ad1  = kron(Paulis["I"],ad);

In [5]:
H_drift = [ ]
H_drives = [
    ad0 * a0,
    ad1 * a1,
    a0+ad0,
    im * (a0-ad0),
    a1+ad1,
    im * (a1-ad1),
    ad0 * a1 + a0 * ad1
]
system = QuantumSystem(H_drives);

In [6]:
PICO_max_iter = 250

# Shape the cost function with weights on states and controls
Q = 100.
R =1e-4
# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .02

theta = 1.05 * pi
temp_trajectories =[]
temp_infidelities=[]
target = excitation(theta)

p = UnitarySmoothPulseProblem(
    system,
    target,
    n_steps,
    Δt;
    a_bound=a_bound,
    dda_bound=dda_bound,
    Q=Q,
    R=R,
    verbose=true,
    hessian_approximation=true,
    pade_order=10,
    free_time=true,
    timesteps_all_equal=false,
    subspace=[1,2,3,4],
    max_iter=PICO_max_iter,
)
solve!(p)
result = copy(p.trajectory)
states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))
println("Infidelity " *string(unitary_infidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target))))
rollout_states = unitary_rollout(result, system; integrator=exp)
ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
println("Rollout Error "*string(maximum(ΔUs)))
println("True Infidelity "*string(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))))

    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14

  71  1.9399297e-05 4.71e-05 2.47e+01  -4.0 4.57e-02    -  1.00e+00 1.00e+00h  1
  72  3.7555855e-05 2.10e-05 2.52e+01  -4.1 3.07e-02    -  1.00e+00 1.00e+00h  1
  73  3.0594263e-05 4.99e-05 2.52e+01  -4.1 3.14e-01    -  1.00e+00 1.25e-01h  4
  74  9.6044991e-05 3.74e-05 2.48e+01  -4.1 3.39e-02    -  1.00e+00 2.50e-01h  3
  75  3.1647308e-05 2.28e-05 2.49e+01  -4.1 2.63e-02    -  1.00e+00 5.00e-01h  2
  76  1.0383899e-04 2.00e-05 2.51e+01  -4.1 5.47e-02    -  1.00e+00 2.50e-01h  3
  77  8.4934600e-05 1.06e-05 2.51e+01  -4.1 2.58e-02    -  1.00e+00 1.00e+00H  1
  78  1.6034320e-05 9.11e-06 2.48e+01  -4.1 3.20e-02    -  1.00e+00 5.00e-01h  2
  79  4.1947087e-05 7.92e-06 2.52e+01  -4.1 5.24e-02    -  1.00e+00 2.50e-01h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  4.6201365e-05 6.66e-06 2.52e+01  -4.1 3.05e-02    -  1.00e+00 2.50e-01h  3
  81  2.8889020e-05 5.86e-06 2.53e+01  -4.1 1.84e-02    -  1.00e+00 1.00e+00h  1
  82  6.6210167e-06 5.05e-06

 164  7.5967359e-06 1.10e-06 2.40e+01  -4.1 7.97e-03    -  1.00e+00 5.00e-01h  2
 165  3.2299917e-05 1.81e-06 2.43e+01  -4.1 1.06e-02    -  1.00e+00 5.00e-01h  2
 166  1.1684764e-05 1.69e-06 2.57e+01  -4.1 9.92e-03    -  1.00e+00 1.25e-01h  4
 167  2.9331697e-05 1.04e-06 2.44e+01  -4.1 4.08e-03    -  1.00e+00 1.00e+00h  1
 168  8.2673339e-06 1.33e-06 2.56e+01  -4.1 2.94e-02    -  1.00e+00 1.25e-01h  4
 169  1.3985021e-05 5.56e-07 2.41e+01  -4.1 4.54e-03    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  1.1046579e-04 5.12e-07 2.58e+01  -4.0 4.57e-03    -  1.00e+00 5.00e-01h  2
 171  1.9331737e-05 5.33e-07 2.63e+01  -4.0 3.11e-03    -  1.00e+00 1.00e+00h  1
 172  4.6796311e-05 6.33e-06 2.60e+01  -4.0 8.26e-02    -  1.00e+00 1.25e-01h  4
 173  5.9545507e-05 4.14e-06 2.40e+01  -4.1 1.61e-02    -  1.00e+00 5.00e-01h  2
 174  3.3748715e-05 3.95e-06 2.42e+01  -4.1 2.08e-02    -  1.00e+00 2.50e-01h  3
 175  3.1805771e-05 7.02e-06

Rollout Error 0.000370819243587025
True Infidelity 0.0001549078943475779
